This tutorial illustrates a few of the fault detection and exclusion capabilities from `algorithms/fde.py`.

In [ ]:
import numpy as np
import gnss_lib_py as glp

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2022/device_gnss.csv --quiet -O "device_gnss.csv"
navdata = glp.AndroidDerived2022("device_gnss.csv")

# FDE: Fault Detection and Exclusion

Several pre-built methods exist for performing fault detection and exclusion and can be accessed through the ``solve_fde()`` function.

## Greedy Euclidean Distance Matrix FDE

The first method is based on "Detection and Exclusion of Multiple Faults using Euclidean Distance Matrices" by Derek Knowles and Grace Gao from the ION GNSS+ 2023 conference.

In [ ]:
result = glp.solve_fde(navdata, method="edm")

After this method runs, a new row is added called "fault_edm" which has a 0 if no fault is predicted, 1 if a fault is predicted, and 2 for an unknown fault status (usually due to lack of necessary columns or information).

In [ ]:
result["fault_edm"]

You can channge the ``threshold`` variable to be more or less sensitive based on false alarm or missed detection system requirements. Greedy EDM FDE has a range for the threshold between 0 and 1 since the detection statistic is normalized to 1. Note that if the threshold is set to the lower limit of zero, faults are detected until only four measurements remain at each timestep.

In [ ]:
result = glp.solve_fde(navdata, method="edm", 
                       threshold=0)
result["fault_edm"]

The ``max_faults`` variable can be changed to apply a maximum number of faults detected at each timestep. In this example, the threshold is still set to zero, but we limit the faults removed with the ``max_faults`` variable.

In [ ]:
result = glp.solve_fde(navdata, method="edm",
                       threshold=0, max_faults=4)
result["fault_edm"]

If the ``remove_outliers`` variable is set, then the outliers and unknown statuses will automatically be removed from the returned ``NavData`` object.

In [ ]:
result = glp.solve_fde(navdata, method="edm",
                       threshold=0, remove_outliers=True)
result["fault_edm"]

## Greedy Residual FDE

This FDE method is based on "Fast multiple fault exclusion with a large number of measurements." by Juan, Blanch, Todd Walter, and Per Enge from the ION GNSS+ 2015 conference.

In [ ]:
result = glp.solve_fde(navdata, method="residual")

After this method runs, a new row is added called "fault_residual" which has a 0 if no fault is predicted, 1 if a fault is predicted, and 2 for an unknown fault status (usually due to lack of necessary columns or information).

In [ ]:
result["fault_residual"]

## Evaluate FDE

The ``evaluate_fde()`` function can be used to create overall metrics on accuracy and timing based on a ground truth fault status row. The below example shows a comparison between the default parameters for greedy EDM and greedy residual FDE.

In [ ]:
edm_metrics, _ = glp.evaluate_fde(navdata, method="edm",
                                  fault_truth_row="MultipathIndicator",
                                  time_fde=True)
for key, value in edm_metrics.items():
    print(key,":",value)

In [ ]:
residual_metrics, _ = glp.evaluate_fde(navdata, method="residual",
                                       fault_truth_row="MultipathIndicator",
                                       time_fde=True)
for key, value in residual_metrics.items():
    print(key,":",value)

Checkout the timing difference between the too methods!

In [ ]:
speed_increase = residual_metrics["timestep_mean_ms"] - edm_metrics["timestep_mean_ms"]
print(f"Greedy EDM is {np.round(speed_increase,1)} milliseconds faster than Greedy Residual on average!")